In [23]:
import advent
from advent.utils import tadd2, tsub2
import numpy as np
advent.scrape(2024, 15)
data = advent.get_lines_doublenewline(15)
grid, directions = data
grid = np.array([[c for c in line] for line in grid])
directions = ''.join(directions)
print(len(np.where(grid == '#')[0]))
print(len(np.where(grid == 'O')[0]))

offsets = {'>': (0, 1), '<': (0, -1), '^': (-1, 0), 'v': (1, 0)}

409
581


In [24]:
def step(grid, direction: tuple[int, int]):
    robot = np.where(grid == '@')
    robot = robot[0][0], robot[1][0]
    new_pos = tadd2(robot, direction)
    while grid[new_pos] != '#' and grid[new_pos] != '.':
        new_pos = tadd2(new_pos, direction)
    if grid[new_pos] == '#': return grid # We cant move this direction
    # Now we want to move robot-(newpos-1) to (robot+1)-(newpos) and set robot to '.'
    assert grid[new_pos] == '.'
    # Nah that's not gonna be that easy...
    #robot_plus_1, new_pos_plus_1 = tadd2(robot, direction), tadd2(new_pos, direction)
    #grid[robot_plus_1[0]:new_pos_plus_1[0], robot_plus_1[1]:new_pos_plus_1[1]] = \
    #    grid[robot[0]:new_pos[0], robot[1]:new_pos[1]]
    i, j = tsub2(new_pos, direction), new_pos
    while j != robot:
        grid[j] = grid[i]
        i, j = tsub2(i, direction), i

    grid[robot] = '.'
    return grid


grid, _ = data
grid = np.array([[c for c in line] for line in grid])
for direction in directions:
    grid = step(grid, offsets[direction])
    #print( f'---- {direction} --- ')
    #for l in grid: print( ''.join(l))
    #assert len(np.where(grid == '#')[0]) == 409
    #assert len(np.where(grid == 'O')[0]) == 581

x, y = np.where(grid == 'O')
result = 0
for i in range(len(x)):
    result += (100 * x[i] + y[i])
print(result)


1349898


In [25]:
# Part 2
# Man this is only day 15... The first week being easy was definitely an illusion

# My approach: For each step, keep a set of elements that need to be moved
# Then just make a copied grid and move the elements one by one

def get_affected(grid, coord: tuple[int, int], direction: tuple[int, int]):
    # If you pushed this block upwards, which blocks would also need to pushed upwards
    if grid[coord] == '#': return None
    if grid[coord] == '.': return set([]) # {coord}
    if grid[coord] == 'O' or grid[coord] == '@':  # O is Only for part 1
        r = get_affected(grid, tadd2(coord, direction), direction)
        return {coord} | r if r is not None else None
    assert grid[coord] in '[]', grid[coord]
    if grid[coord] == ']': otherbox = tadd2(coord, (0, -1))
    if grid[coord] == '[': otherbox = tadd2(coord, (0, 1))
    l = get_affected(grid, tadd2(coord, direction), direction) if tadd2(coord, direction) != otherbox else set()
    r = get_affected(grid, tadd2(otherbox, direction), direction) if tadd2(otherbox, direction) != coord else set()
    if l is None or r is None: return None
    else: return l | r | {coord, otherbox}

def step2(grid, direction):
    robot = np.where(grid == '@')
    robot = robot[0][0], robot[1][0]
    affected = get_affected(grid, robot, direction)
    if affected is None: return grid.copy()
    new_grid = grid.copy()
    new_grid[robot] = '.'
    for a in affected:
        new_grid[tadd2(a, direction)] = grid[a]
        # If this block got pushed but will not be pushed into, it will become empty
        # This bug stumped me for like 20 minute.... D;
        if tsub2(a, direction) not in affected: new_grid[a] = '.'
    return new_grid



In [26]:
data = advent.get_lines_doublenewline(15)
grid2, _ = data
for i, l in enumerate(grid2):
    grid2[i] = l.replace("#", "##").replace(".", "..").replace("O", "[]").replace('@', '@.')
grid2 = np.array([[c for c in line] for line in grid2])    

for direction in directions:
    grid2 = step2(grid2, offsets[direction])
    #print( f'---- {direction} --- ')
    #for l in grid2: print( ''.join(l))

x, y = np.where(grid2 == '[')
result = 0
for i in range(len(x)):
    result += (100 * x[i] + y[i])
print(result)
# 1913350 too high

1376686
